In [ ]:
import pandas as pd
import numpy as np

import json

import matplotlib.pyplot as plt
from sklearn.metrics import *
import time
import os


import warnings
warnings.filterwarnings('ignore')

import wandb
wandb_username = 'andreasabo'
local_username = 'andreasabo'

In [ ]:
# root directory
root_dir = "/home/andreasabo/Documents/HNProject/"
split_file_base = "/home/andreasabo/Documents/HNUltra/"

# data directory on current machine: abhishekmoturu, andreasabo, denizjafari, navidkorhani
data_dir = "/home/" + local_username + "/Documents/HNProject/all_label_img/"
test_data_dir = "/home/" + local_username + "/Documents/HNProject/HNUltra.csv"

# read target df
csv_path = os.path.join(root_dir, "all_splits_1000000.csv")
data_df = pd.read_csv(csv_path, usecols=['image_manu', 'image_ids'])
# data_df = pd.read_csv(csv_path)


label_mapping = {'Other':0, 'Saggital_Right':1, 'Transverse_Right':2, 
                 'Saggital_Left':3, 'Transverse_Left':4, 'Bladder':5}
label_unmapping = {0: 'Other', 1:'Saggital_Right', 2: 'Transverse_Right', 
                   3:'Saggital_Left', 4:'Transverse_Left', 5: 'Bladder'}



In [ ]:
# Compute accuracy and f1 score for each machine 
results_per_fold = {}
for fold in '01234':
    test_data = pd.read_csv(test_data_dir, usecols=["IDs_iter_"+fold, "labels_iter_"+fold, "preds_iter_"+fold])
    test_data["labels_iter_"+fold] = test_data["labels_iter_"+fold].map(label_unmapping)
    test_data["preds_iter_"+fold] = test_data["preds_iter_"+fold].map(label_unmapping)
    joined_df = test_data.set_index("IDs_iter_"+fold).join(data_df.set_index('image_ids'))
    
    machine_manus = list(joined_df.image_manu.unique())
    
    for machine in machine_manus:
        machine_data = joined_df[joined_df.image_manu == machine]
        true_labels = machine_data["labels_iter_"+fold]
        preds = machine_data["preds_iter_"+fold]
        
        if machine in results_per_fold:
            results_per_fold[machine]['acc'].append(accuracy_score(true_labels, preds))
            results_per_fold[machine]['f1'].append(f1_score(true_labels, preds, average='macro'))
        else:
            sub_dict = {}
            sub_dict['acc'] = [accuracy_score(true_labels, preds)]
            sub_dict['f1'] = [f1_score(true_labels, preds, average='macro')]
            sub_dict['support'] = len(true_labels)
            results_per_fold[machine]=sub_dict

# Compute the mean and standard deviation of each machine
mean_results = {'machine':[], 'acc_mean':[], 'acc_std':[], 'f1_mean':[], 'f1_std':[], 'support':[]}
for machine in results_per_fold:
    acc_np = np.asarray(results_per_fold[machine]['acc'])
    f1_np = np.asarray(results_per_fold[machine]['f1'])
    mean_acc, std_acc = acc_np.mean(), acc_np.std()
    mean_f1, std_f1 = f1_np.mean(), f1_np.std()
    
    # Add to dictionary 
    mean_results['machine'].append(machine)
    
    mean_results['acc_mean'].append(mean_acc)
    mean_results['acc_std'].append(std_acc)
    mean_results['f1_mean'].append(mean_f1)
    mean_results['f1_std'].append(std_f1)
    mean_results['support'].append(results_per_fold[machine]['support'])
    
results_df = pd.DataFrame(mean_results)  
print(results_df)
